In [341]:
import numpy as np
# import matplotlib.pyplot as plt
# import math
import pandas as pd

1. Take-off from ship deck carrying the sonar
2. Travel 5 km (on a calm day) over the sea
3. Dive inside the water to a depth of about 500 meters
4. Search the ocean bed and leave the payload at an appropriate location.
5. Rise back to the surface
6. Return to the ship and land.

Relevant payload specifications:
* Weight = 5 kg
* Volume = 0.0025 m3
* Water proof (until 1000m depth)

1. Do the initial sizing of your vehicle rotor using Momentum Theory to decide on tentative
rotor dimensions and power requirement in hover. 

    Also check the rotor’s estimated power requirement under water.

In [342]:
from BET import BETheory
from BEM import BEMTheory

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [358]:
roAir = 1.225   # kg/m3 density of air
roWater = 1000  # kg/m3
g = 9.8         # m/s2

# The airfoil : NACA 2412. Simply because They used that in this paper: file:///home/kushal/Downloads/placement/NACA2412_airfoil_based_method_for_design_and_aerod.pdf
# Mean Chord length is defined below
rBlade = 0.75  # 0.85 # m  Radius of the blade(rough estimate) : The estimate is made looking at this data: https://uav-en.tmotor.com/html/2019/Motor_1016/294.html
R_co = 0.05 # mine was 0.15
massOfStrutcutre= 10# kg this mass of everything other than payload mass.
# angular_vel = 750*2*np.pi/60 
angular_vel = 38
V = 5
# Given
massOfPayload = 5
volOfPayload = 0.0025
b = 4       # no of Blades . Tentatively decided on three
areaBlade = np.pi*(rBlade**2)

total_vol = 0.008 # The payload is gonna be taking a place inside this volume

In [364]:
def vel_power(V, medium='air'):
    if medium == 'air':
        T = (massOfStrutcutre + massOfPayload)*g; ro = roAir
    elif medium == 'water':
        ro = roWater
        T = (massOfStrutcutre + massOfPayload)*g - total_vol*ro*g
        
    ind_vel_MT = (-1*V/2) + np.sqrt(((V/2)**2) + (T/(2*ro*areaBlade)))
    power = T*(V + ind_vel_MT)
    return round(ind_vel_MT, 5) , round(power,4)

In [361]:
print('--'*5, 'MOMENTUM THEORY', '--'*5)
T_air = (massOfStrutcutre + massOfPayload)*g
print(f'T_air = {T_air}')
T_water = (massOfStrutcutre + massOfPayload)*g - total_vol*roWater*g
print(f'T_water = {T_water}')
print('HOVER')
indVel_Hover_MT, powerHover = vel_power(0)
print(f'(Power, Induced Vel) = ({powerHover: .4f}, {indVel_Hover_MT: .4f})')
indVel_Hover_MT,powerHover = vel_power(0,'water')
print(f'(Power, Induced Vel) = ({powerHover: .4f}, {indVel_Hover_MT: .4f})')

indVEl_MT, powerClimb = vel_power(V)
print(f'Climb Vel = {climbVel_MT}')
print(f'(Power, Induced Vel) = ({powerClimb: .4f}, {indVel_MT: .4f})')
# I think the power to go downward in underwater is not a problem(gravity takes care of it).
indVel_MT_water,powerClimb_water = vel_power(V, 'water' )
print(f'(Power, Induced Vel) = ({powerClimb_water: .4f}, {indVel_MT_water: .4f})')

---------- MOMENTUM THEORY ----------
T_air = 147.0
T_water = 68.6
HOVER
(Power, Induced Vel) = ( 856.5580,  5.8269)
(Power, Induced Vel) = ( 9.5573,  0.1393)
Climb Vel = 5
(Power, Induced Vel) = ( 1299.5664,  3.2572)
(Power, Induced Vel) = ( 343.2661,  0.0039)


OBSERVATION

The induced Velocity decreases in CLIMB


In [346]:
inducedVel_MT = np.sqrt(T_air/(2*roAir*areaBlade))
print(f'HOVER Induced Velocity = {inducedVel_MT: .4f}')

HOVER Induced Velocity =  5.8269


Wooh! That is significantly Low, we don't have to worry about water at all.

2. Make computational tools to implement BE Theory and BEM Theory with the Prandtl Tip 
Loss  model  (will  require  iterative  solving). 
    
    The  program  will  be  used  for  design  purpose; 
hence it should be flexible enough to allow setting linear twist, linear taper, and root cut-outs, 
and non-zero climb velocity. 

In [347]:
airfoil_file_path = './xf-naca2412-il-200000.csv'
airfoil_data = pd.read_csv(airfoil_file_path, skiprows= 10)
alpha_cl_cd_data = airfoil_data[['Alpha', 'Cl', 'Cd']]
Cl = np.array(alpha_cl_cd_data['Cl'])
Cd = np.array(alpha_cl_cd_data['Cd'])
alpha = np.array(alpha_cl_cd_data['Alpha'])
Cl_Cd = Cl/Cd
alpha_optimum = alpha[np.argmax(Cl_Cd)]
print(f'AoA for max Cl/Cd = {alpha_optimum}')
# lift_slope = 0.107015*180/np.pi
lift_slope = 2*np.pi
print(f'Lift slope a = {lift_slope:.4f}')

AoA for max Cl/Cd = 6.0
Lift slope a = 6.2832


In [348]:
# This code ensures that No point in r an airfoil experiences Stall
# Esentially how its' done is we give a penalty for AoA > 13 and ask the optimizer to change A,B to minimize that penalty
from scipy.optimize import minimize
stall_angle = 12.5
def calc_AoA(r_ratio, A, B):
    tita = A + B*r_ratio*rBlade
    phi = np.arctan((V + inducedVel_MT)/(angular_vel*r_ratio*rBlade))
    return (tita - phi)*180/np.pi
def objecive(x):
    A,B =x
    aoa_values = [calc_AoA(r, A, B) for r in [0.14,0.3,0.4,0.5,0.6,0.8,0.9,0.96]]
    penalty = 0
    for aoa in aoa_values:
        if aoa > stall_angle or aoa < 0:
            penalty += abs(aoa - stall_angle) + abs(aoa)
    return penalty
initial_guess = [0.9, -1.5]
bounds = [(0.2, 2), (-2,2 )]
result = minimize(objecive, initial_guess, bounds = bounds)
final_A, final_B = result.x

print(round(final_A,4), round(final_B,4))
print(result.fun)

1.1274 -0.7765
32.19279633905401


In [349]:
def linear_twist(r_ratio):
    e,f = final_A, final_B
    # return  e + f*r_ratio*rBlade
    return 0.91 + 0.74*r_ratio*rBlade
def linear_taper(r_ratio):
    # return 0.1 - 0.02*r_ratio*rBlade
    return 0.07-0.02*r_ratio*rBlade
print(f'At a position r/R = 0.2, 0.75 the twist angle is {linear_twist(0.2):.4f}, {linear_twist(0.75):.4f}')
radii = np.linspace(R_co, rBlade, 100)
c_r = [linear_taper(r/rBlade) for r in radii ]
c_mean = np.mean(c_r)
print(f'Mean Chord Length : {c_mean}')

At a position r/R = 0.2, 0.75 the twist angle is 1.0210, 1.3262
Mean Chord Length : 0.06200000000000001


In [350]:
rotorBlade = BETheory(
    angular_vel = angular_vel, no_of_blades= b, chord_mean= c_mean, 
    radius= rBlade, lift_slope=lift_slope, linear_twist=linear_twist, 
    climb_vel=V ,induced_vel= vel_power, root_cutouts=R_co) # I am using vel_power function defined for MT here

In [351]:
rotor = BEMTheory(
    angular_vel = angular_vel, no_of_blades= b, 
    radius= rBlade, lift_slope=lift_slope, linear_twist=linear_twist, 
    climb_vel=V , root_cutouts=R_co, linear_taper=linear_taper)

In [352]:
r = 0.5/rBlade # this is a ratio x/R
print('C_P')
print(f'BET:{rotorBlade.C_P():.4f}, BEMT: {rotor.C_P():.4f}')
print('C_T')
print(f'BET:{rotorBlade.C_T()}, BEMT: {rotor.C_T():.4f}')
print('--'*5)
print(f'BET:  Lamda: {rotorBlade.lamda :.4f}')
print(f'BEMT is a func(r) Lamda,F :{rotor.lamda_tipLoss(r)}')
print(f'Tip Velocity: {angular_vel*rBlade:.4f}')

C_P


BET:0.0296, BEMT: 0.0228
C_T
BET:0.0949, BEMT: 0.1145
----------
BET:  Lamda: 0.3102
BEMT is a func(r) Lamda,F :(0.19729, 0.9783)
Tip Velocity: 28.5000


In [353]:
val = rotor.lamda_tipLoss(r)[0]/r
print(f'{val:.4f}, {np.arctan(val):.4f}')

0.2959, 0.2877


In [354]:
rotor.T_traditional()

197.8107885106021

In [355]:
print('POWER')
print(f'MT:{powerClimb: .3f}, BET:{rotorBlade.P():.3f}, BEMT: {rotor.P():.3f}')
print('Thrust')
print(f'MT:{T_air: .3f}, BET:{rotorBlade.T():.3f}, BEMT: {rotor.T_traditional():.3f}')
print('Induced Vel')
print(f'MT: {vel_power(V)[0]:.3f}, BET:{vel_power(V)[0]:.3f}, BEMT = func(r): {round(rotor.lamda_tipLoss(r)[0]*angular_vel*rBlade - V,5) :.3f}')

POWER
MT: 1299.566, BET:1456.414, BEMT: 1125.223
Thrust
MT: 147.000, BET:163.998, BEMT: 197.811
Induced Vel
MT: 3.841, BET:3.841, BEMT = func(r): 0.623


In [356]:
# the thrust with blade element momentum theory is more than BE. somethings wrong.\
# u have to get lesser thrust with non uniform velocity
#Okay it needs more power clearly. So I suppose the thrust is more here but power required is more too.